# Problem 1: Raw Data Processing

In [ ]:
!pip install pandas pyarrow

In [2]:
import pandas as pd

# Specify your CSV file path
symbol_file = 'C:\Users\Admin\Desktop\airflow\symbols_valid_meta.csv'  

# Read the CSV file
df_symbols = pd.read_csv(symbol_file)

#Subsetting the dataframe with ['Symbol','Security']
df_symbols = df_symbols[['Symbol', 'Security Name']]


In [3]:
import os
import pandas as pd
import pyarrow.parquet as pq

# Define the CSV files directories
etf_dir = 'C:\Users\Admin\Desktop\airflow\etfs'  
stock_dir = 'C:\Users\Admin\Desktop\airflow\stocks'  

# Define a function to load and process the CSV files
def process_files(dir_path):
    dfs = []  # A list to store the dataframes
    for file_name in os.listdir(dir_path):
        if file_name.endswith('.csv'):
            df = pd.read_csv(os.path.join(dir_path, file_name))
            # Extract the required columns
            df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
            dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# Process the ETF and stock data
etf_data = process_files(etf_dir)
stock_data = process_files(stock_dir)

# Concatenate the ETF and stock data

all_data = pd.concat([df_symbols,etf_data, stock_data], ignore_index=True)

# Write the data to Parquet format
all_data.to_parquet('all_data.parquet', engine='pyarrow')


In [4]:
print(all_data)

         Symbol                                      Security Name  \
0             A            Agilent Technologies, Inc. Common Stock   
1            AA                    Alcoa Corporation Common Stock    
2          AAAU                       Perth Mint Physical Gold ETF   
3          AACG  ATA Creativity Global - American Depositary Sh...   
4          AADR                AdvisorShares Dorsey Wright ADR ETF   
...         ...                                                ...   
28159802    NaN                                                NaN   
28159803    NaN                                                NaN   
28159804    NaN                                                NaN   
28159805    NaN                                                NaN   
28159806    NaN                                                NaN   

                Date       Open       High        Low      Close  Adj Close  \
0                NaN        NaN        NaN        NaN        NaN        NaN   
1

# Problem-2 Feature Engineering


In [7]:
data = pd.concat([etf_data, stock_data],ignore_index=True)
# Calculate moving average of Volume
data['vol_moving_avg'] = data['Volume'].rolling(window=30).mean()
# Calculate rolling median of Adj Close
data['adj_close_rolling_med'] = data['Adj Close'].rolling(window=30).median()

In [9]:
print(data['vol_moving_avg'])
print(data['adj_close_rolling_med'])

0                    NaN
1                    NaN
2                    NaN
3                    NaN
4                    NaN
                ...     
28151753    1.368950e+06
28151754    1.381913e+06
28151755    1.405967e+06
28151756    1.420693e+06
28151757    1.455340e+06
Name: vol_moving_avg, Length: 28151758, dtype: float64
0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
             ...  
28151753    24.915
28151754    24.435
28151755    24.435
28151756    24.295
28151757    24.070
Name: adj_close_rolling_med, Length: 28151758, dtype: float64


In [10]:
print(data)

                Date       Open       High        Low      Close  Adj Close  \
0         2013-06-18   4.807330   4.807330   4.807330   4.807330   4.486191   
1         2013-06-19   4.991330   4.991330   4.991330   4.991330   4.657899   
2         2013-06-20   4.916670   4.916670   4.916670   4.916670   4.588226   
3         2013-06-21   5.020000   5.020000   5.020000   5.020000   4.684653   
4         2013-06-24   5.136670   5.136670   5.136670   5.136670   4.793529   
...              ...        ...        ...        ...        ...        ...   
28151753  2020-03-26  23.379999  24.629999  23.379999  24.440001  24.440001   
28151754  2020-03-27  24.000000  24.370001  23.629999  23.980000  23.980000   
28151755  2020-03-30  24.200001  24.850000  23.910000  24.840000  24.840000   
28151756  2020-03-31  24.730000  24.799999  24.030001  24.160000  24.160000   
28151757  2020-04-01  23.760000  24.520000  23.730000  23.969999  23.969999   

             Volume  vol_moving_avg  adj_close_roll

In [11]:
data.to_parquet('data.parquet', engine='pyarrow')

In [ ]:
dataset_train = data[:int(data.shape[0]*0.8)]
dataset_test = data[int(data.shape[0]*0.8):]

#for c in data.columns[data.dtypes == 'object']:    
#    dataset_train[c] = le.fit_transform(dataset_train[c])
#    dataset_test[c] = le.transform(dataset_test[c])

X_train, y_train = dataset_train.iloc[:,:-1],dataset_train.iloc[:,-1]
X_test, y_test = dataset_test.iloc[:,:-1],dataset_test.iloc[:,-1]
    
#scaler = MinMaxScaler(feature_range=(0,1))
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)
print(dataset_train.shape,dataset_test.shape)
#print(dataset_test)


# Create a XGBRegressor model
model = XGBRegressor()


# Train the model
model.fit(X_train, y_train)

# Make predictions on test data
y_pred = model.predict(X_test)

#Save the resulting model to disk.
np.savetxt('Results.csv',y_pred ,delimiter=',')


# Calculate the variance
print ('metrics.explained_variance_score : ', metrics.explained_variance_score(y_test, y_pred))

# Calculate the mean_absolute_error
print ('metrics.mean_absolute_error : ', metrics.mean_absolute_error(y_test, y_pred))

# Calculate the metrics.mean_squared_error
print ('metrics.mean_squared_error : ', metrics.r2_score(y_test, y_pred))